# Урок 8. Рекуррентные нейронные сети RNN, LSTM, GRU

## Домашнее задание.

Данные берем `'отзывы за лето'`

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и/или (RNN -> CNN)
4. сделать выводы что получилось лучше

In [1]:
!pip install stop_words
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32898 sha256=2be692053fd959d35d0ec8768704767c54d4051b623ea144e7b8a4b5ca07df1c
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=7e04327cb4e4d9c6680e643d790fff27d9a4347cd2ff71c0bdb88900c6e566ae
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


###Загрузка данных

In [2]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

In [3]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [4]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw] #долгая Каноническая форма слова (например, форма единственного числа, именительного падежа для существительных)
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [5]:
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
#import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, Conv2D, GlobalMaxPool1D, concatenate, Flatten, add, MaxPool1D, RepeatVector
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Masking, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, TimeDistributed, AveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [7]:
tokenizer = Tokenizer(num_words=None,
                     filters='#$%&()*+-<=>@[\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train) #Текст в последовательности
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length) #Эта функция преобразует список (длиной num_samples) последовательностей (списков целых чисел) в двумерный массив Numpy
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [8]:
y_train = df_train['class'].values
y_val = df_val['class'].values

### RNN Model

In [9]:
model_name = 'RNN'
model = Sequential()
model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=30, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(SimpleRNN(32, recurrent_dropout=0.2, return_sequences=True)) #возвращать всю последовательность выходных данных для каждого элемента (по одному вектору на каждый шаг)
model.add(SimpleRNN(32, recurrent_dropout=0.2))#отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться


tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name,
    write_graph=False, update_freq=100, profile_batch=0)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 79s 212ms/step - loss: 0.6302 - accuracy: 0.6158 - val_loss: 0.5128 - val_accuracy: 0.7420
Epoch 2/10
319/319 [==============================] - 46s 144ms/step - loss: 0.4009 - accuracy: 0.8241 - val_loss: 0.5155 - val_accuracy: 0.7497


In [10]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 9ms/step - loss: 0.5220 - accuracy: 0.7466


Test score: 0.522027313709259
Test accuracy: 0.7465943694114685


### CNN -> RNN Model


In [11]:
model_name = 'CNN -> RNN'
model = Sequential()

model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=64, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(Conv1D(128, 3, activation='relu', padding="same"))
model.add(SimpleRNN(64, recurrent_dropout=0.2)) #отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name,
    write_graph=False, update_freq=100, profile_batch=0)



In [12]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 50s 135ms/step - loss: 0.5779 - accuracy: 0.6774 - val_loss: 0.4963 - val_accuracy: 0.7539
Epoch 2/10
319/319 [==============================] - 24s 75ms/step - loss: 0.3181 - accuracy: 0.8677 - val_loss: 0.5310 - val_accuracy: 0.7478


### RNN -> CNN Model

In [13]:
model_name = 'RNN -> CNN'
model = Sequential()

model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=64, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(SimpleRNN(64, recurrent_dropout=0.2, return_sequences=True)) #отбрасывание линейных значений повторов 0.2
# model.add(RepeatVector(32))
model.add(Conv1D(128, 3, activation='relu', padding="same"))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name,
    write_graph=False, update_freq=100, profile_batch=0)


In [14]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 46s 132ms/step - loss: 0.6175 - accuracy: 0.6312 - val_loss: 0.5054 - val_accuracy: 0.7468
Epoch 2/10
319/319 [==============================] - 25s 78ms/step - loss: 0.3834 - accuracy: 0.8396 - val_loss: 0.5159 - val_accuracy: 0.7544


#LSTM Models

In [15]:
model_name = 'LSTM'
model = Sequential()

model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=30, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(LSTM(64, recurrent_dropout=0.2)) #отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name,
    write_graph=False, update_freq=100, profile_batch=0)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 64s 189ms/step - loss: 0.5547 - accuracy: 0.7092 - val_loss: 0.4949 - val_accuracy: 0.7532
Epoch 2/10
319/319 [==============================] - 52s 162ms/step - loss: 0.3287 - accuracy: 0.8645 - val_loss: 0.5521 - val_accuracy: 0.7467


In [16]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 14ms/step - loss: 0.5888 - accuracy: 0.7356


Test score: 0.58878093957901
Test accuracy: 0.7356169819831848


#### Получим предсказание тональности твита

In [17]:
df_test.text.values

array(['тектоника рельефсамый ужасный мир мучение',
       'ходить запускать шар желание насна получиться хрен они',
       'хотеть лето ради направить ноготь яркий лак', ...,
       'rt killgayslut yournovocaine привеееть муд черта сэмми',
       'настроение вроде нормальный плакать хотеться',
       'зайти сон девчонкампока мыть посудунастя фоткаться httptcokinexudtuh'],
      dtype=object)

In [18]:
#предобработка текста для предсказания модели
def preprocess_text_for_prediction(txt):
  txt = preprocess_text(txt)
  txt = tokenizer.texts_to_sequences([txt])
  matr_txt = pad_sequences(txt, maxlen=training_length)
  mart_txt = matr_txt.reshape(1, -1)
  return matr_txt


In [19]:
model.predict(
    [preprocess_text_for_prediction('тектоника рельефсамый ужасный мир мучение')],
    batch_size=None,
    )

1/1 [==============================] - 0s 281ms/step


array([[0.05922808]], dtype=float32)

In [20]:
model.predict(
    [preprocess_text_for_prediction('Люблю смотреть на звезды')],
    batch_size=None,
)

1/1 [==============================] - 0s 26ms/step


array([[0.6788312]], dtype=float32)

In [21]:
model.predict(
    [preprocess_text_for_prediction('Великолепно!')],
    batch_size=None,
)

1/1 [==============================] - 0s 43ms/step


array([[0.9239022]], dtype=float32)

### Bidirectional LSTM

In [22]:
model_name = 'Bidirectional LSTM'
inputs = Input(shape=(X_train.shape[1],))

x =     Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True)(inputs)

xbi = Bidirectional(LSTM(15, return_sequences=True))(x) #Двунаправленный
x = add([x, xbi])

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)



model = Model(inputs=inputs, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
early_stopping=EarlyStopping(monitor='val_loss')

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name,
    write_graph=False, update_freq=100, profile_batch=0)


history_1 = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 78s 212ms/step - loss: 0.5482 - accuracy: 0.7109 - val_loss: 0.4853 - val_accuracy: 0.7583
Epoch 2/10
319/319 [==============================] - 52s 162ms/step - loss: 0.2944 - accuracy: 0.8798 - val_loss: 0.5430 - val_accuracy: 0.7475


In [24]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 17ms/step - loss: 0.5639 - accuracy: 0.7380


Test score: 0.563946008682251
Test accuracy: 0.7380416989326477


#CNN Model


In [25]:
model_name = 'CNN'
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=100,
              trainable=True,
              mask_zero=True))
#model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
early_stopping=EarlyStopping(monitor='val_loss')

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name,
    write_graph=False, update_freq=100, profile_batch=0)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 35s 105ms/step - loss: 0.5380 - accuracy: 0.7165 - val_loss: 0.4797 - val_accuracy: 0.7611
Epoch 2/10
319/319 [==============================] - 13s 41ms/step - loss: 0.2532 - accuracy: 0.8969 - val_loss: 0.5443 - val_accuracy: 0.7486


In [27]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 4ms/step - loss: 0.6056 - accuracy: 0.7372


Test score: 0.6055634021759033
Test accuracy: 0.7372040748596191


Все модели показали относительно одинаковые результаты
